In [1]:
import tensorflow as tf
#tf.enable_eager_execution()

# Having trouble getting GPU to be detected properly
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
sess = tf.Session(config=config)

In [2]:
from pycalphad import Database, Model, variables as v
from pycalphad.codegen.sympydiff_utils import build_functions
from sympy import lambdify
import numpy as np

dbf = Database('alzn_mey.tdb')
mod = Model(dbf, ['AL', 'ZN'], 'LIQUID')

In [3]:
mod.variables

[T, LIQUID0AL, LIQUID0ZN]

In [4]:
cy_func = build_functions(mod.GM, mod.variables, include_grad=False)
%time cy_func.kernel # trigger JIT

CPU times: user 116 ms, sys: 7.31 ms, total: 123 ms
Wall time: 1.74 s


<function wrapper_module_148f0b1a_1243_4df3_b425_932e799a6443.autofunc_c>

In [5]:
%timeit cy_func.kernel(np.array([[300., 0.5, 0.5]]), np.array([0.]))

The slowest run took 14.26 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.07 µs per loop


In [6]:
%%timeit
tf_func = lambdify(mod.variables, mod.GM, modules='tensorflow')

10 loops, best of 3: 43.3 ms per loop


In [23]:
tf_func = lambdify(mod.variables, mod.GM, modules='tensorflow')

In [28]:
%%timeit
x1 = tf.placeholder("float", None)
x2 = tf.placeholder("float", None)
x3 = tf.placeholder("float", None)
result = tf_func(x1, x2, x3)

1 loop, best of 3: 161 ms per loop


In [ ]:
x1 = tf.placeholder("float", None)
x2 = tf.placeholder("float", None)
x3 = tf.placeholder("float", None)
result = tf_func(x1, x2, x3)

In [27]:
%timeit sess.run(result, {x1: 300., x2: 0.5, x3: 0.5})

The slowest run took 4.49 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 448 µs per loop
